# 1 Installation des librairies de Sumo  

In [2]:
import traci
import sumolib
import os
import sys
import gymnasium as gym
import random
import numpy as np
from pettingzoo import ParallelEnv
from gymnasium import spaces

# Création de l'environnement custom multi agent pettingzoo

In [6]:
fichier_config = r".\data\networks\2way-single-intersection\single-intersection.sumocfg"
sumoBinary_gui = r"C:\Program Files (x86)\Eclipse\Sumo\bin\sumo-gui.exe"
sumoBinary = r"C:\Program Files (x86)\Eclipse\Sumo\bin\sumo.exe"

sumoCmd = [sumoBinary, r"-c", fichier_config, "-Q","true"]

In [3]:
action_size = 3
state_size = 2

class FeuParFeu(ParallelEnv):
    metadata = {
        "name": "environnementfpf",
    }

    def __init__(self ,render_mode="human",with_gui = False):
        self.with_gui = with_gui
        self.feu6 = "r"
        self.feu0 = "r"
        self.timestep = None
        self.possible_agents = ["feu6", "feu0"]
        self.duree_simulation = None
        self.done = False
        self.traffic_signals = ["t"]
        self.actions = {cle : None for cle in self.possible_agents}
        print(self.actions)
        self.ts_ids = [0, 1, 2, 3, 4, 5, 6, 7]
        self.feu6_action = None
        if with_gui:
            sumoBinary = r"C:\Program Files (x86)\Eclipse\Sumo\bin\sumo-gui.exe"
        else:
            sumoBinary = r"C:\Program Files (x86)\Eclipse\Sumo\bin\sumo.exe"
        self.render_mode = render_mode
        self.sumoCmd = [sumoBinary, r"-c", fichier_config, "-Q","true"]
        traci.start(self.sumoCmd)
        self.controlled_links = traci.trafficlight.getControlledLinks('t')
        self.duree_simulation = int(traci.simulation.getEndTime())
        print(self.duree_simulation)
        self.red_duration = {cle : 0 for cle in self.possible_agents}
        self.green_duration = {cle : 0 for cle in self.possible_agents}
        self.orange_duration = {cle : 0 for cle in self.possible_agents}
        self.dict_observation_space = { }
        self.timestep = None
        self.actionprec = {cle : traci.trafficlight.getRedYellowGreenState('t')[int(cle[3:])]for cle in self.possible_agents}
        print('self.actionprec', self.actionprec)

    def reset(self, seed=None, options=None):
        traci.close()
        traci.start(self.sumoCmd)
        self.agents = self.possible_agents.copy()
        self.timestep = 0
        self.feu6 = "r"
        self.feu0 = "r"

        observations = {
            a: (
                 traci.lane.getLastStepHaltingNumber(self.controlled_links[6][0][0]),
                 traci.lane.getLastStepHaltingNumber(self.controlled_links[0][0][0])
            )
            for a in self.agents
        }
        # Get dummy infos. Necessary for proper parallel_to_aec conversion
        infos = {a: {} for a in self.agents}
        state = {a: np.reshape(observations[a], [1, state_size]) for a in self.agents}
        return observations, infos

    def step(self, actions):
        print(actions)

        for feu, couleur_prec in self.actionprec.items():
            if couleur_prec == 'r':

                if self.red_duration[feu] >= 120:
                    couleur = "G"
                    self.red_duration[feu] = 0

                else:
                    if actions[feu] == 0:
                         couleur = "r"
                         self.red_duration[feu] += 1
                    else:
                        couleur = "G"
                        self.green_duration[feu] += 1
                        self.red_duration[feu] = 0


            elif couleur_prec == 'y':
                if self.orange_duration[feu] <= 3:
                    couleur = 'y'
                    self.orange_duration[feu] += 1
                else:
                    couleur = "r"
                    self.red_duration[feu] += 1
                    self.orange_duration[feu] = 0

            else :
                if self.green_duration[feu] <= 6:

                    couleur = 'G'
                    self.green_duration[feu] += 1

                else:
                    if actions[feu] == 0:
                         couleur = "G"
                         self.green_duration[feu] += 1
                    else:
                        couleur = "y"
                        self.orange_duration[feu] += 1
                        self.green_duration[feu] = 0
            traci.trafficlight.setLinkState('t', int(feu[3:]), couleur)
            self.actionprec[feu] = couleur


        # Check termination conditio


        terminations = {a: False for a in self.agents}
        rewards = {a: 0 for a in self.agents}
        if traci.trafficlight.getRedYellowGreenState('t')[6] == "G":
            rewards["feu6"] = 1  # Récompense pour feu6 lorsque feu6 est vert
        elif traci.trafficlight.getRedYellowGreenState('t')[6] == "r":
            rewards["feu6"] = -1  # Récompense négative pour feu6 lorsque feu6 est rouge
        elif traci.trafficlight.getRedYellowGreenState('t')[6] == "y":
            rewards["feu6"] = 0  # Pas de récompense pour feu6 lorsque feu6 est jaune

        if traci.trafficlight.getRedYellowGreenState('t')[0] == "G":
            rewards["feu0"] = 1  # Récompense pour feu0 lorsque feu0 est vert
        elif traci.trafficlight.getRedYellowGreenState('t')[0] == "r":
            rewards["feu0"] = -1  # Récompense négative pour feu0 lorsque feu0 est rouge
        elif traci.trafficlight.getRedYellowGreenState('t')[0] == "y":
            rewards["feu0"] = 0

    # Check truncation conditions (overwrites termination conditions)
        truncations = {a: False for a in self.agents}
        if self.timestep > self.duree_simulation:
            terminations = {a: True for a in self.agents}
        self.timestep += 1

    # Get observations

        observations = {
            a: (
                 traci.lane.getLastStepHaltingNumber(self.controlled_links[6][0][0]),
                 traci.lane.getLastStepHaltingNumber(self.controlled_links[0][0][0])
            )
            for a in self.agents
        }

    # Get dummy infos (not used in this example)
        infos = {a: {} for a in self.agents}

        if any(terminations.values()) or all(truncations.values()):
            self.agents = []
            self.done = True
            traci.close()

        traci.simulationStep()

        return observations, rewards, self.done, truncations, infos


    def render(self):
        # traci.simulationStep()
        pass

    def observation_space(self,agent):
        """Return the observation space of a traffic signal.

        Only used in case of single-agent environment.
        """
        #return self.traffic_signals[self.ts_ids[0]].observation_space
        #return self.traffic_signals[self.ts_ids[6]].observation_space
        #return spaces.MultiDiscrete([2*1]*2)
        return spaces.discrete.Discrete(1)

    def action_space(self, agent):
        return gym.spaces.discrete.Discrete(2)




1.   Actions : Discret
2.   API parallèle : Non
1.   Contrôle manuel : Non
1.   Agents : agents= ['feu0', 'feu6']
2.   nb Agents : 2
2.   Forme d'action : Discrete(2)
3.   Valeurs d'actions : [0,1]
1.   Forme d'observation : Discrete(1)
2.   Valeurs d'observation : [0,+inf[









In [9]:
import numpy as np

# Deux tableaux 2D
a = np.array([1])

b = np.array([2])

# Empiler les tableaux le long d'un nouvel axe
stacked_array = np.stack((a, b), axis=0)

print("Tableau a:")
print(a)
print("\nTableau b:")
print(b)
print("\nTableaux empilés le long du nouvel axe (axis=0):")
print(stacked_array)
print(stacked_array.shape)


Tableau a:
[1]

Tableau b:
[2]

Tableaux empilés le long du nouvel axe (axis=0):
[[1]
 [2]]
(2, 1)


In [13]:
expanded_arrays = [np.array([0], dtype=np.int16), np.array([0], dtype=np.int16), np.array([0], dtype=np.int16), np.array([0], dtype=np.int16), np.array([0], dtype=np.int16), np.array([0], dtype=np.int16), np.array([0], dtype=np.int16), np.array([0], dtype=np.int16), np.array([0], dtype=np.int16), np.array([0], dtype=np.int16), np.array([0], dtype=np.int16)]
out = np.zeros(shape=(11,1))
axis = 0
np.concatenate(expanded_arrays, axis=axis, out=out)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [14]:
out_v1 = np.zeros(shape=(11,1))
print(f"out_v1 : {out_v1}")

out_vrien = np.zeros(shape=(11,))
print(f"out_vrien : {out_vrien}")

out_v1 : [[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]
out_vrien : [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [4]:
import numpy as np 
l = [1, 2, 3]
chiffre = [5]

np.array(l + chiffre)

array([1, 2, 3, 5])

In [4]:
env = FeuParFeu()

{'feu6': None, 'feu0': None}
100000
self.actionprec {'feu6': 'G', 'feu0': 'G'}


In [5]:
traci.getConnection()

In [11]:
traci.trafficlight.getControlledLinks('t')[2][0][0]

        

'n_t_1'

In [63]:
couleurs = traci.trafficlight.getRedYellowGreenState('t')
print(f"couleurs : {couleurs}")
print(f"len(couleurs) : {len(couleurs)}")

couleurs : GGrrrrGGrrrr
len(couleurs) : 12


In [59]:
traci.close()
traci.trafficlight.setLinkState

FatalTraCIError: Not connected.

In [1]:
class Person():
    def __init__(self, age) -> None:
        self.age = age
    
person = Person(21)
id(person)

1772858529424

In [15]:
traci.start(sumoCmd, label='0')

(21, 'SUMO 1.19.0')

In [16]:
traci.getConnection(label='0').close()

## Demarage d'une simulation de 3600 secondes

In [8]:
import random

# Réinitialiser l'environnement
observations, infos = env.reset()
total_reward_feu6 = 0  # Variable pour stocker la somme des récompenses pour feu6
total_reward_feu0 = 0  # Variable pour stocker la somme des récompenses pour feu0

# Boucle principale pour interagir avec l'environnement
for _ in range(3600):
    # Exécuter une action dans l'environnement
    liste_f6 = [0, 1, 2]  # Liste d'actions pour feu6
    liste_f0 = [0, 1, 2]  # Liste d'actions pour feu0
    action_f6 = random.choice(liste_f6)  # Choix aléatoire de l'action pour feu6
    action_f0 = random.choice(liste_f0)  # Choix aléatoire de l'action pour feu0 différente de celle pour feu6
    actions = {"feu6": action_f6, "feu0": action_f0}  # Actions pour les deux feux
    observations, rewards, terminations, truncations, infos = env.step(actions)
    total_reward_feu6 += rewards["feu6"]  # Ajouter la récompense du feu6 à la somme totale
    total_reward_feu0 += rewards["feu0"]  # Ajouter la récompense du feu0 à la somme totale

    # Afficher les observations et les récompenses
    print("Observations:", observations)
    print("Rewards:", rewards)

# Afficher la somme totale des récompenses pour chaque agent
print("Total rewards for feu6:", total_reward_feu6)
print("Total rewards for feu0:", total_reward_feu0)

# Fermer l'environnement
env.close()


Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
{'feu6': 0, 'feu0': 1}
Observations: {'feu6': (0, 0), 'feu0': (0, 0)}
Rewards: {'feu6': -1, 'feu0': 0}
{'feu6': 1, 'feu0': 2}
Observations: {'feu6': (1, 0), 'feu0': (1, 0)}
Rewards: {'feu6': 1, 'feu0': 0}
{'feu6': 2, 'feu0': 0}
Observations: {'feu6': (0, 1), 'feu0': (0, 1)}
Rewards: {'feu6': 1, 'feu0': -1}
{'feu6': 2, 'feu0': 0}
Observations: {'feu6': (0, 2), 'feu0': (0, 2)}
Rewards: {'feu6': 1, 'feu0': -1}
{'feu6': 2, 'feu0': 2}
Observations: {'feu6': (0, 2), 'feu0': (0, 2)}
Rewards: {'feu6': 1, 'feu0': 1}
{'feu6': 2, 'feu0': 1}
Observations: {'feu6': (0, 3), 'feu0': (0, 3)}
Rewards: {'feu6': 1, 'feu0': 1}
{'feu6': 0, 'feu0': 0}
Observations: {'feu6': (0, 2), 'feu0': (0, 2)}
Rewards: {'feu6': 1, 'feu0': 1}
{'feu6': 0, 'feu0': 2}
Observations: {'feu6': (0, 1), 'feu0': (0, 1)}
Rewards: {'feu6': 1, 'feu0': 1}
{'feu6': 1, 'feu0': 1}
Observations: {'feu6': (0, 1), 'feu0': (0, 1)}
Rewards: {'feu6': 0, 'feu0': 1}
{

# test dalgorithme

In [3]:
import numpy as np

# Créer un tableau de forme (11, 2)
arr1 = np.random.rand(11, 2)
print("arr1 :", arr1)  # Affiche (11, 2)

# Créer un tableau de forme (2,)
arr2 = np.array([1, 2])
print("arr2 :", arr2)  # Affiche (2,)

# Essayer d'additionner les deux tableaux
result = arr1 + arr2
print(f"result : {result}")

arr1 : [[0.32655595 0.42915219]
 [0.6899888  0.71739561]
 [0.83388607 0.54610347]
 [0.41507792 0.00127269]
 [0.5094375  0.58180225]
 [0.53639926 0.16335464]
 [0.18403829 0.84363587]
 [0.67087328 0.56502705]
 [0.24922747 0.59204597]
 [0.82817309 0.7460124 ]
 [0.16808247 0.983821  ]]
arr2 : [1 2]
result : [[1.32655595 2.42915219]
 [1.6899888  2.71739561]
 [1.83388607 2.54610347]
 [1.41507792 2.00127269]
 [1.5094375  2.58180225]
 [1.53639926 2.16335464]
 [1.18403829 2.84363587]
 [1.67087328 2.56502705]
 [1.24922747 2.59204597]
 [1.82817309 2.7460124 ]
 [1.16808247 2.983821  ]]


In [ ]:
!pip install stable_baselines3
import gymnasium as gym
!pip install supersuit

In [ ]:
from stable_baselines3.ppo import CnnPolicy
from stable_baselines3.ppo import MlpPolicy
from stable_baselines3.ppo import MultiInputPolicy
from stable_baselines3.common.vec_env import VecMonitor
from stable_baselines3 import PPO
import supersuit as ss

In [ ]:
traci.close()

In [ ]:
env = FeuParFeu()
vec_env = ss.pettingzoo_env_to_vec_env_v1(env)
concat_env = ss.concat_vec_envs_v1(vec_env,2 , num_cpus=1, base_class="stable_baselines3")
sb3env = VecMonitor( concat_env)

{'feu6': None, 'feu0': None}
 Retrying in 1 seconds
3600
self.actionprec {'feu6': 'r', 'feu0': 'G'}


AttributeError: 'FeuParFeu' object has no attribute 'ts'

In [ ]:
model = PPO("MlpPolicy", concat_env, verbose=1)
model.learn(total_timesteps=2)

In [ ]:
 mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=1)